In [3]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END

In [2]:
!pip install -qU langchain-anthropic langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.4 MB/s eta 0:00:00


In [12]:

class AgentState(TypedDict):
  number1: int
  number2: int
  operation1: str

  number3: int
  number4: int
  operation2: str

  finalNumber1: int
  finalNumber2: int



In [13]:
def adder(state: AgentState) -> AgentState:
  """ This adds two numbers """
  state['finalNumber1'] = state['number1'] + state['number2']

  return state

def substractor(state: AgentState) -> AgentState:
  """ This subtracts two numbers """
  state['finalNumber1'] = state['number1'] - state['number2']

  return state

def adder2(state: AgentState) -> AgentState:
  """ This adds two numbers """
  state['finalNumber2'] = state['number3'] + state['number4']

  return state

def substractor2(state: AgentState) -> AgentState:
  """ This subtracts two numbers """
  state['finalNumber2'] = state['number3'] - state['number4']

  return state


In [14]:
def decide_next_node(state: AgentState) -> AgentState:
  if state['operation1'] == "+":
    return "addition_operation"
  elif state['operation1'] == '-':
    return "substraction_operation"

def decide_next_node2(state: AgentState) -> AgentState:
  if state['operation2'] == "+":
    return "addition_operation"
  elif state['operation2'] == '-':
    return "substraction_operation"

In [20]:
graph = StateGraph(AgentState)
graph.add_node("add_node",adder)
graph.add_node("substract_node",substractor)

graph.add_node("add_node2",adder2)
graph.add_node("substract_node2",substractor2)

graph.add_node("router",lambda state: state) # This is a passthrough function
graph.add_node("router2",lambda state: state) # This is a passthrough function

graph.add_edge(START, "router")
graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        "addition_operation": "add_node",
        "substraction_operation": "substract_node"
    }
)
graph.add_conditional_edges(
    "router2",
    decide_next_node2,
    {
        "addition_operation": "add_node2",
        "substraction_operation": "substract_node2"
    }
)
graph.add_edge("add_node","router2")
graph.add_edge("substract_node","router2")

graph.add_edge("add_node2",END)
graph.add_edge("substract_node2",END)


app = graph.compile()


In [21]:
inital_state = AgentState(number1 = 10, operation1="-", number2=5, number3=7, number4=2, operation2="+",finalNumber1=0, finalNumber2=0)
print(app.invoke(inital_state))

{'number1': 10, 'number2': 5, 'operation1': '-', 'number3': 7, 'number4': 2, 'operation2': '+', 'finalNumber1': 5, 'finalNumber2': 9}
